In [2]:
import os
import json
import requests
import folium
import pandas as pd
import geopandas as gpd
from dotenv import load_dotenv

load_dotenv()


True

In [49]:
gdb_path = "D:\Downloads\Download_Geo-data-viz-data_2388450\poi_5285468\poi_5285468.gpkg"
layer_name = 'Points of Interest 2023_06'
gdf_places = gpd.read_file(gdb_path, layer=layer_name)
gdf_places.crs = 'epsg:27700'

In [90]:
gdb_path = "D:\Downloads\Download_Geo-data-viz-data_2388450\codepoint-poly_5285467\\two_letter_pc_code\cb.zip"
gdf_post = gpd.read_file(gdb_path)[['POSTCODE', 'geometry']].to_crs(epsg=4326).reset_index(drop=True)

In [73]:
food_gdf = gdf_places[gdf_places['classname'].isin(['Pubs, Bars and Inns', 'Cafes, Snack Bars and Tea Rooms', ])][['name', 'classname', 'street_name', 'postcode', 'url', 'geometry']]\
                     .to_crs(epsg=4326).reset_index(drop=True)

# Step 3: Convert to GeoJSON (assuming your gdf is not empty)
geojson = food_gdf.to_json()
food_gdf

,name,classname,street_name,postcode,url,geometry
0,Thoroughbreads,"Cafes, Snack Bars and Tea Rooms",Orwell Furlong,CB4 0PP,www.thoroughbreads.com,POINT (0.15151 52.23172)
1,The Orchard Tea Garden,"Cafes, Snack Bars and Tea Rooms",Mill Way,CB3 9ND,www.theorchardteagarden.co.uk,POINT (0.09645 52.17756)
2,The Senate,"Pubs, Bars and Inns",St Marys Passage,CB2 3PQ,www.thesenatebistro.com,POINT (0.11813 52.20501)
3,Savino's Coffee Bar Italiano,"Cafes, Snack Bars and Tea Rooms",St Andrews Street,CB2 3BZ,www.savinos.co.uk,POINT (0.12330 52.20436)
4,Regent News & Snacks,"Cafes, Snack Bars and Tea Rooms",Regent Street,CB2 1AW,None,POINT (0.12659 52.20038)
...,...,...,...,...,...,...
260,Gastronome,"Cafes, Snack Bars and Tea Rooms",Bridge Street,CB2 1UF,None,POINT (0.11841 52.20873)
261,Fancett's,"Cafes, Snack Bars and Tea Rooms",Mill Road,CB1 2BD,None,POINT (0.13854 52.19929)
262,Mina Steakhouse,"Pubs, Bars and Inns",Brookgate,CB1 2FF,None,POINT (0.13404 52.19267)
263,Heavenly Desserts,"Cafes, Snack Bars and Tea Rooms",Christs Lane,CB1 1NP,None,POINT (0.12302 52.20512)


In [127]:
post_gdf = gdf_post[gdf_post['POSTCODE'].str.startswith('CB')].dissolve(by='POSTCODE', as_index=False).reset_index(drop=True)
post_gdf

,POSTCODE,geometry
0,CB1 1AL,"POLYGON ((0.13582 52.20771, 0.13608 52.20773, ..."
1,CB1 1AN,"POLYGON ((0.13582 52.20771, 0.13578 52.20758, ..."
2,CB1 1AP,"POLYGON ((0.12977 52.20592, 0.12964 52.20594, ..."
3,CB1 1AQ,"POLYGON ((0.13582 52.20771, 0.13587 52.20783, ..."
4,CB1 1AR,"POLYGON ((0.12579 52.20765, 0.12576 52.20758, ..."
...,...,...
10241,CB9 9SG,"POLYGON ((0.43083 52.09330, 0.43070 52.09317, ..."
10242,CB9 9SH,"POLYGON ((0.42913 52.09276, 0.42913 52.09275, ..."
10243,CB9 9SJ,"POLYGON ((0.42920 52.09491, 0.42917 52.09502, ..."
10244,CB9 9SL,"MULTIPOLYGON (((0.43103 52.09467, 0.43096 52.0..."


In [128]:
joined_gdf = gpd.sjoin(food_gdf, post_gdf, how="inner", op='within').reset_index(drop=True)
joined_gdf

c:\Users\us\anaconda3\envs\detectree2-env\lib\site-packages\IPython\core\interactiveshell.py:3488: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,name,classname,street_name,postcode,url,geometry,index_right,POSTCODE
0,The Orchard Tea Garden,"Cafes, Snack Bars and Tea Rooms",Mill Way,CB3 9ND,www.theorchardteagarden.co.uk,POINT (0.09645 52.17756),5750,CB3 9ND
1,The Senate,"Pubs, Bars and Inns",St Marys Passage,CB2 3PQ,www.thesenatebistro.com,POINT (0.11813 52.20501),2084,CB2 3PQ
2,The Senate,"Cafes, Snack Bars and Tea Rooms",St Marys Passage,CB2 3PQ,None,POINT (0.11813 52.20501),2084,CB2 3PQ
3,Savino's Coffee Bar Italiano,"Cafes, Snack Bars and Tea Rooms",St Andrews Street,CB2 3BZ,www.savinos.co.uk,POINT (0.12330 52.20436),2059,CB2 3BU
4,Cafe Milano,"Cafes, Snack Bars and Tea Rooms",St Andrews Street,CB2 3BZ,None,POINT (0.12280 52.20454),2059,CB2 3BU
...,...,...,...,...,...,...,...,...
252,Tall Trees,"Pubs, Bars and Inns",Lensfield Road,CB2 1EN,www.thesnugbar.co.uk,POINT (0.12293 52.19793),1950,CB2 1EN
253,Chesterton Corner,"Cafes, Snack Bars and Tea Rooms",High Street,CB4 1NW,None,POINT (0.14232 52.21763),5991,CB4 1NW
254,The Red Mug,"Cafes, Snack Bars and Tea Rooms",Chesterton Road,CB4 1EP,None,POINT (0.12706 52.21451),5862,CB4 1DS
255,Sarina's,"Cafes, Snack Bars and Tea Rooms",Cherry Hinton Road,CB1 7BS,None,POINT (0.14161 52.18862),569,CB1 7BS


In [129]:
counts_df = joined_gdf.groupby(['classname', 'POSTCODE']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
merged_df = pd.merge(post_gdf, counts_df, how='inner', on='POSTCODE')
merged_df

,POSTCODE,geometry,classname,counts
0,CB1 1BG,"MULTIPOLYGON (((0.13253 52.20378, 0.13251 52.2...","Pubs, Bars and Inns",1
1,CB1 1BH,"POLYGON ((0.13178 52.20308, 0.13153 52.20328, ...","Cafes, Snack Bars and Tea Rooms",1
2,CB1 1DB,"MULTIPOLYGON (((0.13395 52.20592, 0.13452 52.2...","Pubs, Bars and Inns",1
3,CB1 1DG,"POLYGON ((0.13366 52.20591, 0.13398 52.20573, ...","Cafes, Snack Bars and Tea Rooms",2
4,CB1 1DJ,"POLYGON ((0.13184 52.20610, 0.13180 52.20616, ...","Cafes, Snack Bars and Tea Rooms",1
...,...,...,...,...
182,CB5 8JE,"POLYGON ((0.14347 52.20974, 0.14351 52.20975, ...","Pubs, Bars and Inns",2
183,CB5 8LD,"POLYGON ((0.14704 52.21125, 0.14670 52.21105, ...","Cafes, Snack Bars and Tea Rooms",1
184,CB5 8ST,"POLYGON ((0.17340 52.22030, 0.17340 52.22029, ...","Pubs, Bars and Inns",1
185,CB5 8SX,"POLYGON ((0.16986 52.22839, 0.16989 52.22845, ...","Pubs, Bars and Inns",1


In [133]:
import folium
import json
from folium.features import GeoJsonTooltip

# Create map
m = folium.Map(location=[52.205320488806095, 0.11825440531707056], zoom_start=15)

# Step 5: Add the GeoJSON to the Map
# folium.GeoJson(geojson, ).add_to(m)

# Function to create a stylized popup HTML
def create_popup_html(name, address, url):
    return f"""
    <div>
        <h4 style="margin-bottom:0;"><a href="{url}" target="_blank">{name}</a></h4>
        <p style="margin-top:0;"><strong>Address:</strong> {address}</p>
    </div>
    """

# Function to create a stylized tooltip HTML
def create_tooltip_html(name):
    return f"<div style='font-size: 12px; font-weight: bold;'>{name}</div>"

# Create choropleth
choropleth = folium.Choropleth(
    geo_data=merged_df,
    name='choropleth',
    data=merged_df,
    columns=['POSTCODE', 'counts'],
    key_on='feature.properties.POSTCODE',
    fill_color='YlGnBu',
    fill_opacity=1,
    line_opacity=0.5,
    legend_name='Counts'
)

# Add tooltip to the choropleth
choropleth.geojson.add_child(
    GeoJsonTooltip(
        fields=['classname', 'POSTCODE', 'counts'],
        aliases=['Type: ', 'Postcode: ', 'Count: '],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
)

# Add choropleth to the map
choropleth.add_to(m)

# Feature group for points
points_feature_group = folium.FeatureGroup(name='Coffee Shops & Bars').add_to(m)

# Add points to the feature group
for feature in json.loads(geojson)['features']:
    classname = feature['properties']['classname']
    name = feature['properties']['name']
    address = feature['properties']['street_name']
    url = feature['properties']['url']
    url = '' if url is None else url
    geom = feature['geometry']['coordinates']
    
    marker = folium.Marker(
        location=[geom[1], geom[0]],
        popup=folium.Popup(html=create_popup_html(name, address, url), max_width=300),
        tooltip=folium.Tooltip(text=create_tooltip_html(name)),
        icon=folium.Icon(color='orange', icon='coffee', prefix='fa') if classname == 'Cafes, Snack Bars and Tea Rooms' else folium.Icon(color='darkblue', icon='glass')
    )
    marker.add_to(points_feature_group)

# Add Layer Control to toggle the layers
folium.LayerControl(collapsed=False).add_to(m)

# Save and show map
# m.save('/mnt/data/folium_map_with_layer_control.html')
m  # Display the map if using Jupyter Notebooks
